In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import os
from PIL import Image
from sklearn.model_selection import GridSearchCV
from pickle import dump, load
from sklearn.model_selection import ParameterGrid


In [3]:
# images_folder = 'images'

# X = []
# y = []

# for subdir, dirs, files in os.walk(images_folder):
#     for file in files:
#         if file.endswith(('jpg', 'jpeg', 'png')):
#             img_path = os.path.join(subdir, file)
#             label = os.path.basename(subdir)
            
#             image = Image.open(img_path).convert('L')
#             image = image.resize((48, 48))
#             X.append(np.array(image).flatten())
#             y.append(label)

# with open("svc_standardscaler_gridsearch_xy_dump.pkl", "wb") as f:
#     dump((X,y), f, protocol=5)

In [4]:
with open("svc_standardscaler_gridsearch_xy_dump.pkl", "rb") as f:
    X,y = load(f)

In [5]:
X = np.array(X)
y = np.array(y)

In [6]:
# Normalize image data
X = X / 255.0  # Normalize pixel values to [0, 1]

In [7]:
# Encode labels as integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [8]:
pca = PCA(n_components=100)  # Reduce to 100 components
X_reduced = pca.fit_transform(X)

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reduced)

In [10]:
with open("svc_standardscaler_gridsearch_normalizers_dump.pkl", "wb") as f:
    dump((label_encoder, pca, scaler), f, protocol=5)

In [11]:
X_train , X_test, y_train, y_test =  train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [12]:
# Grid Search
param_grid = {
    'C': [1, 10, 100],  # Smaller range for C
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['scale'], 
    'degree': [1, 2, 3],  
}

all_combinations = list(ParameterGrid(param_grid))

for combination in all_combinations:
    print(combination)
    
    svc_model = SVC(**combination)
    svc_model.fit(X_train, y_train)

    # Make predictions
    y_pred = svc_model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


print("\nTotal combinations:", len(all_combinations))

with open("svc_model_standardscaler_grisearch_pca_dump.pkl", "wb") as f:
    dump(svc_model, f, protocol=5)

print("Best Parameters:", svc_model.best_params_)

{'C': 1, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.69
Classification Report:
              precision    recall  f1-score   support

       angry       0.72      0.54      0.62      1963
     disgust       1.00      0.44      0.61       217
        fear       0.71      0.56      0.63      2064
       happy       0.68      0.86      0.76      3590
     neutral       0.65      0.68      0.66      2504
         sad       0.61      0.66      0.64      2417
    surprise       0.86      0.75      0.80      1600

    accuracy                           0.69     14355
   macro avg       0.75      0.64      0.67     14355
weighted avg       0.70      0.69      0.69     14355

{'C': 1, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}
Accuracy: 0.38
Classification Report:
              precision    recall  f1-score   support

       angry       0.31      0.12      0.17      1963
     disgust       0.00      0.00      0.00       217
        fear       0.26      0.11      0.15      

C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

Accuracy: 0.28
Classification Report:
              precision    recall  f1-score   support

       angry       0.14      0.09      0.11      1963
     disgust       0.00      0.00      0.00       217
        fear       0.12      0.07      0.08      2064
       happy       0.38      0.60      0.46      3590
     neutral       0.26      0.25      0.26      2504
         sad       0.21      0.20      0.20      2417
    surprise       0.33      0.30      0.31      1600

    accuracy                           0.28     14355
   macro avg       0.20      0.22      0.20     14355
weighted avg       0.25      0.28      0.26     14355

{'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.69
Classification Report:
              precision    recall  f1-score   support

       angry       0.72      0.54      0.62      1963
     disgust       1.00      0.44      0.61       217
        fear       0.71      0.56      0.63      2064
       happy       0.68      0.86      0.76      3590

C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

Accuracy: 0.21
Classification Report:
              precision    recall  f1-score   support

       angry       0.09      0.09      0.09      1963
     disgust       0.00      0.00      0.00       217
        fear       0.09      0.07      0.08      2064
       happy       0.33      0.43      0.37      3590
     neutral       0.17      0.15      0.16      2504
         sad       0.15      0.14      0.14      2417
    surprise       0.24      0.23      0.24      1600

    accuracy                           0.21     14355
   macro avg       0.15      0.16      0.15     14355
weighted avg       0.19      0.21      0.20     14355

{'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

       angry       0.84      0.86      0.85      1963
     disgust       0.98      0.91      0.94       217
        fear       0.86      0.85      0.86      2064
       happy       0.91      0.93      0.92      359

C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shane_1\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

Accuracy: 0.21
Classification Report:
              precision    recall  f1-score   support

       angry       0.10      0.10      0.10      1963
     disgust       0.00      0.00      0.00       217
        fear       0.09      0.08      0.08      2064
       happy       0.33      0.42      0.37      3590
     neutral       0.17      0.14      0.16      2504
         sad       0.16      0.15      0.15      2417
    surprise       0.24      0.24      0.24      1600

    accuracy                           0.21     14355
   macro avg       0.16      0.16      0.16     14355
weighted avg       0.19      0.21      0.20     14355

{'C': 100, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

       angry       0.85      0.86      0.85      1963
     disgust       0.99      0.94      0.96       217
        fear       0.86      0.88      0.87      2064
       happy       0.92      0.93      0.92      35

AttributeError: 'SVC' object has no attribute 'best_params_'

In [ ]:
# param_grid = {
#     'C': [1, 10, 100],
#     'kernel': ['rbf', 'poly', 'sigmoid'],
#     'gamma': ['scale'],
#     'degree': [1, 2, 3],
# }


# print("Performing Grid Search...")
# svc_model = GridSearchCV(
#     SVC(), param_grid, cv=3, verbose=2, n_jobs=2, refit=True, return_train_score=True
# )
# svc_model.fit(X_train, y_train)

# # Extract all results and display them
# results_df = pd.DataFrame(svc_model.cv_results_)
# results_df = results_df.sort_values(by="mean_test_score", ascending=False)

# # Display top results
# pd.set_option('display.max_colwidth', None)
# print("\nAll Parameter Combinations and Accuracies:")
# print(results_df[['params', 'mean_test_score', 'std_test_score']])

# # Save the best model
# best_model = svc_model.best_estimator_
# with open("svc_model_best_standardscaler_grisearch_pca_dump.pkl", "wb") as f:
#     dump(best_model, f)

# # Display the best parameters
# print("\nBest Parameters:", svc_model.best_params_)




# print("\nEvaluating the Best Model...")
# # Make predictions
# y_pred = best_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# # Evaluate the model
# print(f"Accuracy on Test Set: {accuracy:.2f}")
# print("Classification Report:")
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Performing Grid Search...
Fitting 3 folds for each of 27 candidates, totalling 81 fits

All Parameter Combinations and Accuracies:
                                               params  mean_test_score  \
25  {'C': 100, 'degree': 3, 'gamma': 'scale', 'ker...         0.749125   
18  {'C': 100, 'degree': 1, 'gamma': 'scale', 'ker...         0.745886   
21  {'C': 100, 'degree': 2, 'gamma': 'scale', 'ker...         0.745886   
24  {'C': 100, 'degree': 3, 'gamma': 'scale', 'ker...         0.745886   
16  {'C': 10, 'degree': 3, 'gamma': 'scale', 'kern...         0.745833   
12  {'C': 10, 'degree': 2, 'gamma': 'scale', 'kern...         0.745189   
15  {'C': 10, 'degree': 3, 'gamma': 'scale', 'kern...         0.745189   
9   {'C': 10, 'degree': 1, 'gamma': 'scale', 'kern...         0.745189   
22  {'C': 100, 'degree': 2, 'gamma': 'scale', 'ker...         0.688013   
3   {'C': 1, 'degree': 2, 'gamma': 'scale', 'kerne...         0.617809   
0   {'C': 1, 'degree': 1, 'gamma': 'scale', 'kerne...  

In [19]:
# Make Prediciton
best_knn = svc_model.best_estimator_
best_knn
y_pred = best_knn.predict(X_test)
y_pred

array([4, 4, 3, ..., 4, 5, 2])

In [20]:
# Evaluate the model
print("Classification Report:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Classification Report:
Accuracy: 0.8938349007314524
              precision    recall  f1-score   support

       angry       0.90      0.85      0.87      1963
     disgust       0.92      0.92      0.92       217
        fear       0.91      0.86      0.89      2064
       happy       0.90      0.93      0.92      3590
     neutral       0.85      0.90      0.87      2504
         sad       0.89      0.87      0.88      2417
    surprise       0.94      0.93      0.93      1600

    accuracy                           0.89     14355
   macro avg       0.90      0.89      0.90     14355
weighted avg       0.89      0.89      0.89     14355

Accuracy: 0.89


In [18]:
# Here you can replace pickle with joblib or cloudpickle
# from pickle import dump
# with open("filename.pkl", "wb") as f:
#     dump(clf, f, protocol=5)